In [1]:
from datetime import date, timedelta
import os
import pickle
import sys
import pandas as pd
sys.path.append("..")

In [2]:
# %pip install -U pandas

In [3]:
name = "rel_diff_ci"
df = pickle.load(
    open(os.path.join("..", "tests", "cache", f"{name}.p"), "rb" )
)
# df.head(2)

In [5]:
from abautomator.visualizer import visualizer

from bokeh.io import output_notebook
from bokeh.plotting import show

output_notebook()

p = visualizer.get_figure(df)

Loading BokehJS ...

In [6]:
show(p)